# Encrypt and decrypt a picture with AES

As the title say, I'm going to show you how I managed to encrypt and decrypt a picture with AES.
I'm using python3 and for AES encryption the library pycryptodome.

## Find a mode for AES

AES have different mode for encryption (ie: ECV, CBC ...). The first question I asked myself is what could be the optimal mode for encrypting a picture ? <br/>
The differences between modes is the way we encrypt. Each mode have an encryption scheme which lead to different caracteristics. <br/>
For instance the simpliest mode to use, **ECB**, allow you to encrypt data fast and well. But the disadvantage of this method is a lack of diffusion. **ECB** encrypts identical plaintext blocks into identical ciphertext blocks, it does not hide data patterns well as you can see in the following picture:

![ecb](https://upload.wikimedia.org/wikipedia/commons/c/c0/Tux_ECB.png)

This is the logo of linux after being encrypted with **ECB** mode. You can notice patterns and guess the picture. <br/>
No need to explain why I can't chose this mode ! <br/> <br/>

To solve this issue, we can use **CBC** mode that grant us a good repartition of the pixels with no patterns which is a good thing. 
This mode doens't handle parallelizable encryption which made the encryption longer. But our use do not require such an effiency so we don't really mind about this. 
So we are going to keep **CBC** mode for our script<br/> <br/>


## How CBC work

A quick reminder on how **CBC** work. <br/>

Ciphertext Block Chaining is a mode of operation where each plaintext block gets XOR-ed with the previous ciphertext block prior to encryption. <br/>
A block is a chunk of data with a size of 16 bytes. We need to split our ciphertext in many 16 bytes block as we can. If the plaintext is too short to fit exactly 16 bytes, we use a function called pad() that fill the remaining bytes of random data.

In order to start the encryption, we need initiate a value called the Initialization Vector (iv). This value will be generated by our secret key that we can chose or generate randomly <br/>
For security purpose, I advice you to generate it randomly. <br/> <br/>

This is a scheme of CBC encryption: <br/>
![cbc](https://www.researchgate.net/profile/Rhouma-Rhouma/publication/215783767/figure/fig1/AS:394138559238144@1470981363092/Cipher-block-chaining-CBC-mode-encryption.png)

So lets resume how it work: 
- First we generate or chose by ourself a secret key
- The IV is generated from our key
- We pad the plaintext so we have the guaranty to have a 16 bytes block
- Finnaly, we encrypt our data

This is my implementation using the pycryptodome library:

In [16]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

import os


def generate_secret_key():
    return os.urandom(16)   #Here the function urandom() is generating a random byte number of 16 bytes

def encrypt(data, key):
    cipher = AES.new(key, AES.MODE_CBC)                            
    ciphertext = cipher.encrypt(pad(data, AES.block_size))         
    return ciphertext, cipher.iv

key = generate_secret_key()
ct, iv = encrypt(b"Hello World", key)
print (f"Ciphertext: {ct} | IV: {iv}")

Ciphertext: b'\xe9\xf4\x05T&\xaf+s\xaa\xb5\xde\x1f\xa2\x15\xd2\xa7' | IV: b'\x8c\xc5\\\xc6\xee\x14k\x9e\\\x0e\x98\x02\xf6XM9'


To proceed to the decryption of the data, we need to undo the operation and give the correct key and IV: <br/>

In [17]:
def decrypt(data, key, iv):
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(data), AES.block_size)

print (decrypt(ct, key, iv))

b'Hello World'


## Extract the data from a picture


We know how to encrypt and decrypt with AES but how can we encrypt a picture ? As you may know, a picture is made of pixels, represented by a number, that refers to a colour. <br/>
The task is simple: we need to extract those pixels, encrypt them and re-inject the data into the picture.

To do so we can use the PIL library. W


In this example I'm using a picture that I've download: the windows XP background.

In [14]:
from PIL import Image


def get_pixel(filename):
    img = Image.open(filename)
    return img.tobytes()

def inject_pixel(data, filename):
    img = Image.open(filename)
    data = img.frombytes(data)
    img.putpixel(data)

filename = "linux.jpeg"
data = get_pixel(filename)
inject_pixel(data, filename)

SystemError: src/_imaging.c:205: bad argument to internal function

We got a byte string that contains our picture and the injection rewrite the picture correctly. We have now every component we need, lets proceed to a complete simulation to verify if the script really work:

# References
- Pycrypotodome AES: https://pycryptodome.readthedocs.io/en/latest/src/cipher/aes.html <br/>
- A list of AES modes: https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation
- How to choose an AES encryption mode https://stackoverflow.com/questions/1220751/how-to-choose-an-aes-encryption-mode-cbc-ecb-ctr-ocb-cfb